In [1]:
import pandas as pd

In [2]:
from tqdm import tqdm

In [3]:
df=pd.read_csv("/home/umair123/pytorchproject/llama_project/train.csv")

In [4]:
df=df.drop(labels=range(1001,12177),axis=0,inplace=False)
df=df.reset_index()

In [5]:
df.columns

Index(['index', 'Case Name', 'Input', 'Output', 'Label', 'Count',
       'Decision_Count'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           1002 non-null   int64 
 1   Case Name       1002 non-null   object
 2   Input           1002 non-null   object
 3   Output          1002 non-null   object
 4   Label           1002 non-null   int64 
 5   Count           1002 non-null   int64 
 6   Decision_Count  1002 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 54.9+ KB


In [7]:
df.head()

,index,Case Name,Input,Output,Label,Count,Decision_Count
0,0,"KAMLESH Vs. UNION OF INDIA THROUGH SECRETARY, ...","S. ABDUL NAZEER, J. 1. In this appeal, appella...",0[ds]7. It is clear from the order of appointm...,0,1253,564
1,1,KANWAR PAL SINGH Vs. THE STATE OF UTTAR PRADESH,"SANJIV KHANNA, J. 1. Leave granted. 2. Kanwar ...",1[ds]5. We find the submission of the appellan...,1,4065,2268
2,2,Manke Ram Vs. State of Haryana,"Santosh Hegde, J. 1. The incident which gives ...",1[ds]6. Having perused the material on record ...,1,1616,708
3,3,Kr. Jyoti Sarup and Another Vs. Board of Reven...,"S.K. Das, J. 1. These two appeals and the conn...","0[ds]7. We are in agreement with the view, exp...",0,2440,784
4,4,"Commissioner Of Income-Tax, Kerala Vs. Gemini ...","Shah, J. 1. Two persons-Walter and Ramasubramo...",1[ds]That case can have no application to the ...,1,3317,722


In [8]:
def preprocess_case(text):
    max_tokens=1000
    tokens=text.split(' ')
    num_tokens_to_extract=min(max_tokens,len(tokens))
    text1 = ' '.join(tokens[-num_tokens_to_extract:len(tokens)])
    return text1

In [9]:
for i, row in df.iterrows():
    input=row['Input']
    input=preprocess_case(input)
    df.at[i,'Input']=input

In [10]:
def format_instruction(sample, inst):
    return f"""### Instruction:
{inst}

### Input:
{sample['Input']}

## Response:
{sample['Output'].split('[ds]')[0]}

### Explanation:
{preprocess_case(sample['Output'].split('[ds]')[1])}
"""
#prediction

# def format_instruction(sample, inst):
# 	return f"""### Instruction:
# {inst}

# ### Input:
# {sample['Input']}

# ### Response:
# {sample['Output'].split('[ds]')[0]}
# """

In [11]:
df_ins = pd.read_csv("instruction_decision.csv")

In [12]:
df_ins.head()

,Instructions,Instructions_Exp
0,Analyze the case proceeding and predict whethe...,"First, predict whether the appeal in case proc..."
1,Based on the information in the case proceedin...,Determine the likely decision of the case (acc...
2,Review the case details and predict the decisi...,Predict the outcome of the case proceeding (1 ...
3,Considering the arguments and evidence in case...,Evaluate the case proceeding to forecast the c...
4,Examine the details of the case proceeding and...,Ascertain if the court will uphold (1) or dism...


In [13]:
len(df_ins)

16

In [14]:
df.iloc[16]

index                                                            16
Case Name         Unni Krishnan, J.P. and others etc. etc Vs. St...
Input             and / or affiliation is not a matter of course...
Output            1[ds]143. It is thus well established by the d...
Label                                                             1
Count                                                         46753
Decision_Count                                                 5498
Name: 16, dtype: object

In [15]:
df_ins['Instructions_Exp'][15]

"Hypothesize the court's verdict (affirmation (1) or negation (0) of the appeal), and then clarify this hypothesis by interpreting significant sentences from the case proceeding."

In [16]:
import random
random.seed(15)
text = []
for i,row in tqdm(df.iterrows()):
  random_index = random.randint(0, len(df_ins) - 1)
  t = format_instruction(df.iloc[i],df_ins['Instructions_Exp'][random_index])
  # t = format_instruction(df.iloc[i],df_ins['Instructions'][random_index])
  text.append(t)

1002it [00:00, 12306.60it/s]


In [17]:
df.loc[:, "text"] = text

In [18]:
random_state = 42

# Calculate the number of rows for validation set (10%)
val_size = int(0.5 * len(df))

df_val = df.sample(n=val_size, random_state=random_state)

df_train = df.drop(df_val.index)

df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

In [19]:
df_train.to_csv("train_ft.csv",index=False)
df_val.to_csv("val_ft.csv",index=False)